# Nodejs MNIST model Deployment

 * Wrap a nodejs tensorflow model for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
 
## Depenencies

 * [S2I](https://github.com/openshift/source-to-image)
 * pip install grpcio-tools
 * node (version>=8.11.0)
 * npm

## Train locally using npm commands


In [ ]:
!npm install
!npm start


Training creates a model.json file and a weights.bin file which is utilized for prediction

## Prediction using REST API on the docker container

In [ ]:
!s2i build . seldonio/seldon-core-s2i-nodejs:0.1 node-s2i-mnist-model:0.1

In [ ]:
!docker run --name "nodejs_mnist_predictor" -d --rm -p 5000:5000 node-s2i-mnist-model:0.1

In [ ]:
!cd ../../../wrappers/testing && make build_protos

Send some random features that conform to the contract

In [ ]:
!python ../../../wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p -t

In [ ]:
!docker rm nodejs_mnist_predictor --force